# Direct job execution

This notebook illustrates how to use LM Buddy as a library to run jobs directly on the host machine.

Jobs are executed in the following manner:
- Construct an instance of the `lm_buddy.LMBuddy` class
- Construct an instance of your desired job configuration
- Execute a job via the `LMBuddy.finetune` or `LMBuddy.evaluate` methods

**Warning**: This workflow is still considered experimental.
Some jobs depend on external services (e.g., W&B, Ray cluster) and host-machine GPU resources,
and may not work without a properly configured local environment.

## Imports

In [ ]:
from lm_buddy import LMBuddy
from lm_buddy.jobs.configs import (
    FinetuningJobConfig,
    FinetuningRayConfig,
    LMHarnessJobConfig,
    LMHarnessEvaluationConfig,
)
from lm_buddy.integrations.huggingface import (
    HuggingFaceRepoConfig,
    AutoModelConfig,
    TextDatasetConfig,
    TrainerConfig,
    AdapterConfig,
)
from lm_buddy.integrations.wandb import WandbRunConfig

## Finetuning

In [ ]:
# Base model to finetune from HuggingFace
model_config = AutoModelConfig(
    load_from=HuggingFaceRepoConfig(repo_id="distilgpt2"),
)

# Text dataset for finetuning
dataset_config = TextDatasetConfig(
    load_from=HuggingFaceRepoConfig(repo_id="imdb"),
    split="train[:100]",
    text_field="text",
)

# HuggingFace trainer arguments
trainer_config = TrainerConfig(
    max_seq_length=256,
    per_device_train_batch_size=8,
    learning_rate=1e-4,
    num_train_epochs=1,
    logging_strategy="steps",
    logging_steps=1,
    save_strategy="epoch",
    save_steps=1,
)

# LORA adapter settings
adapter_config = AdapterConfig(
    peft_type="LORA",
    task_type="CAUSAL_LM",
    r=8,
    lora_alpha=16,
    lora_dropout=0.2,
)

# Define tracking for finetuning run
tracking_config = WandbRunConfig(
    name="example-finetuning",
    project="lm-buddy-examples",  # Update to your project name
    entity="mozilla-ai",  # Update to your entity name
)

# Ray train settings
ray_config = FinetuningRayConfig(
    use_gpu=False,  # Change to True if GPUs are available on your machine
    num_workers=2,
)

# Full finetuning config
finetuning_config = FinetuningJobConfig(
    model=model_config,
    dataset=dataset_config,
    trainer=trainer_config,
    adapter=adapter_config,
    tracking=tracking_config,
    ray=ray_config,
)

In [ ]:
# Run the job
buddy = LMBuddy()
buddy.finetune(finetuning_config)

## Evaluation

In [ ]:
# Define the model to be evaluated
# In this case, loading directly a pretrained model from HuggingFace
model_config = AutoModelConfig(
    load_from=HuggingFaceRepoConfig(repo_id="distilgpt2"),
)

# Define evaluation tasks and settings
evaluation_config = LMHarnessEvaluationConfig(
    tasks=["hellaswag"],
    limit=10,  # Only run 10 samples per task. Remove for a real run.
    num_fewshot=5,
)

# Define tracking for eval run
tracking_config = WandbRunConfig(
    name="example-lm-harness",
    project="lm-buddy-examples",  # Update to your project name
    entity="mozilla-ai",  # Update to your entity name
)

# Full lm-harness job config
lm_harness_config = LMHarnessJobConfig(
    model=model_config,
    evaluation=evaluation_config,
    tracking=tracking_config,
)

In [ ]:
# Run the job
buddy = LMBuddy()
eval_results = buddy.evaluate(lm_harness_config)